In [1]:
from config import *
import geopandas as gpd
import pandas as pd

barrios = gpd.read_file(map_barrios)
comisarias = pd.read_csv(map_comisarias)
centros_salud = gpd.read_file(map_ctros_salud)
transporte = pd.read_csv(map_transporte_urb)

comisarias = gpd.GeoDataFrame(comisarias, geometry=gpd.points_from_xy(comisarias.Longitud, comisarias.Latitud))
transporte = gpd.GeoDataFrame(transporte, geometry=gpd.points_from_xy(transporte.longitud, transporte.latitud))

In [2]:
import pandas as pd
from config import *
import os

train = pd.DataFrame()
for file in os.listdir(path_unify):
  if file.__contains__("-09-"): train = train._append(pd.read_excel(path_unify + file))
  
train = train.loc[train["coordX"].notna()]

In [5]:
from math import radians, sin, cos, sqrt, atan2

def calcular_distancia(coord1, coord2):
    radio_tierra = 6371.0

    latitud1, longitud1 = radians(coord1[0]), radians(coord1[1])
    latitud2, longitud2 = radians(coord2[0]), radians(coord2[1])

    dlat = latitud2 - latitud1
    dlon = longitud2 - longitud1

    a = sin(dlat / 2)**2 + cos(latitud1) * cos(latitud2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distancia = radio_tierra * c

    return distancia

def calcular_distancia_minima(coord, coords):
    distancias = []
    for c in coords:
        distancias.append(calcular_distancia(coord, c))
    return min(distancias)

def calcular_distancia_minima_comisarias(coord):
    return calcular_distancia_minima(coord, comisarias[["Latitud", "Longitud"]].values)

def calcular_distancia_minima_tu(coord):
    return calcular_distancia_minima(coord, transporte[["latitud", "longitud"]].values)

def calcular_distancia_minima_cs(coord):
    return calcular_distancia_minima(coord, centros_salud[["Latitud", "Longitud"]].values)

def calcular_distancia_centro(coord):
    return calcular_distancia(coord, (-31.4201, -64.1888))

In [6]:
df = train.copy()

df["comisariaCercana"] = df[["coordX", "coordY"]].apply(calcular_distancia_minima_comisarias, axis=1)
df["transporteCercano"] = df[["coordX", "coordY"]].apply(calcular_distancia_minima_tu, axis=1)
df["saludCercana"] = df[["coordX", "coordY"]].apply(calcular_distancia_minima_cs, axis=1)
df["distanciaCentro"] = df[["coordX", "coordY"]].apply(calcular_distancia_centro, axis=1)

In [ ]:
df.to_csv("../Data/test.csv", index=False)